In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import codecs
import numpy as np
import random
import pickle
import os
import math
import string

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

from torch.autograd import Variable
from torch.optim import lr_scheduler
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split




Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
import os 

x = []
y = []

from os import listdir
from os.path import isfile, isdir, join


def str_full_to_half(in_str):
    out_str = []
    for char in in_str:
        inside_code = ord(char)
        if inside_code == 0x3000 or inside_code == 12288 or char == string.whitespace:  # 全形空格直接轉換
            out_str.append(' ')
        elif inside_code >= 65281 and inside_code <= 65374:
            inside_code -= 0xfee0
            out_str.append(chr(inside_code))
        else:
            out_str.append(char)

    return ''.join(out_str)


paths = listdir('/kaggle/input/corpus-2014/2014/2014/')


# files
split_s = []

features = []
labels = []
count = 0
for path in paths:
    if len(labels) > 3000:
        break
    if '.DS_Store' in path:
        continue
    files = listdir('/kaggle/input/corpus-2014/2014/2014/{}'.format(path))
   
    for file in files:
        if '.txt' not in file:
            continue
        split_s = []
        label = []
        feature = []
        with open('/kaggle/input/corpus-2014/2014/2014/{}/{}'.format(path, file), 'r', encoding='utf-8-sig') as f:
            for sentence in f:
                s = sentence.split()
                for token in s:
                    if len(token)>0 and '/' in token:
                        words = str_full_to_half(token).split('/')
                        is_name = 0
                        if 'nr' == words[1]:
                            is_name = 1
                
                        for word in words[0]:
                            feature.extend([word])
                            label.extend([is_name])
                        
            if len(feature) > 64:
                features.append(feature[:64])
                labels.append(np.array(label[:64]))
            else:
                features.append(feature)
                labels.append(np.array(label))
#             labels.append(np.array(label))
            count += 1
            
            if count % 1000 == 0:
                print(count)


1000
2000
3000


In [4]:
for i in labels:
    i.resize((64), refcheck=False)

In [5]:
## keras tokenizer
tokenizer = text.Tokenizer(filters='', lower=True)
tokenizer.fit_on_texts(features)

## padding 操作

MAX_LEN = 64
x_train = tokenizer.texts_to_sequences(features)
# x_test = tokenizer.texts_to_sequences(x_test)

x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
# x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN, padding='post')

In [6]:
### 將資料轉為torch格式
x_train_torch = torch.tensor(x_train, dtype=torch.long).cuda()
y_train_torch = torch.tensor(np.array(labels), dtype=torch.long).cuda()
# x_test_torch = torch.tensor(x_test, dtype=torch.long)
# y_test_torch = torch.tensor(np.array(y_test), dtype=torch.long)

In [7]:
from torch.utils.data import Dataset, DataLoader,TensorDataset

train_data = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)

# test_data = torch.utils.data.TensorDataset(x_test_torch, y_test_torch)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)



In [8]:

# 是否使用GPU
is_cuda = torch.cuda.is_available()


class TextCNN(nn.Module):
    def __init__(self, embedding_matrix, drop_prob=0.5, embedding_dim=256, n_classes=2):
        super(TextCNN, self).__init__()
        
        self.max_feature = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
        self.embedding = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.dropout = nn.Dropout(drop_prob)
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.Tanh())

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(256),
            nn.Tanh())

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm1d(256),
            nn.Tanh())

        self.conv1d3 = nn.Conv1d(in_channels=256 * 3, out_channels=512, kernel_size=3, padding=1)
        self.conv1d1 = nn.Conv1d(in_channels=512, out_channels=128, kernel_size=1)
        self.fc = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        
        x3 = self.conv3(x)
        x5 = self.conv5(x)
        x7 = self.conv7(x)

        x = torch.cat((x3, x5, x7), dim=1)
        x = self.dropout(x)
        x = self.conv1d3(x)
        x = self.conv1d1(x)

#         x = x.view(x.size(0), -1)
        x = x.view(-1,x.size(1))
        logits = self.fc(x)

        return F.softmax(logits, dim=-1)

# 為了解決類別間不均衡的焦距損失
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float)): self.alpha = t.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = t.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()




In [9]:
def build_matrix(word_index, embedding_index):

    embedding_matrix = np.zeros((len(word_index) + 1, 256))    
    
    for word, i in word_index.items():
        
        if word in embedding_index:
            embedding_matrix[i] = embedding_index[word]
            continue
        else:
            embedding_matrix[i] = np.random.standard_normal(256)
            
    return embedding_matrix

In [10]:

with open('/kaggle/input/hanzi2vec/hanzi2vec256.pkl','rb') as fp:
    chars = pickle.load(fp)
embedding_matrix = build_matrix(tokenizer.word_index, chars)



In [11]:
embedding_matrix.shape

(3212, 256)

In [12]:
import time

EPOCH = 100

net = TextCNN(embedding_matrix)
net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

for epoch in range(EPOCH):
    print('epoch {}'.format(epoch+1))
    net.train()
    
    avg_loss = 0.0
    
    for data in train_loader:
        inputs, labels = data
        pred = net(inputs)
#         loss = nn.CrossEntropyLoss()(pred, labels.view(-1)) + FocalLoss(gamma=2)(pred, labels.view(-1))
#         labels = np.argmax(labels.cpu().detach().numpy(), -1).astype(np.float32)
        loss =FocalLoss(gamma=5)(pred, labels.view(-1))
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
            
        avg_loss += loss.item()
        
    
    net.eval()
    
    for i, data in enumerate(train_loader):
        
        if i == 3:
            break
            
        inputs, labels = data
        idx = np.random.choice(np.random.permutation(len(inputs)) , 4)
        test_word = inputs[idx]
        test_labels = labels[idx]
        
        for w, l in zip(test_word, test_labels):
            news_content = []
            pred_name = []
            true_name = []
            p = net(w.view(1, -1)).cpu()
            p = [np.argmax(i.detach().numpy()) for i in p]
            for token, label, pr in zip(w.cpu().numpy(), l.cpu().numpy(), p):
                if token == 0:
                    continue
                news_content.append(tokenizer.index_word[token])
                if label == 1:
                    true_name.append(tokenizer.index_word[token])
                if pr == 1:
                    pred_name.append(tokenizer.index_word[token])

            print('新聞內容：{}'.format(''.join(news_content)))
            print('偵測之名字 ： {}'.format(''.join(pred_name)))
            print('實際之名字 ： {}'.format(''.join(true_name)))
            print('---'*30)
            
    avg_val_loss = 0.0
#     for data in test_loader:
#         inputs, labels = data
        
#         idx = np.random.choice(np.random.permutation(len(inputs)) , 3)
#         test_word = inputs[idx]
#         test_labels = labels[idx]
        
#         for w, l in zip(test_word, test_labels):
#             news_content = []
#             pred_name = []
#             true_name = []
#             p = net(w.view(1, -1))
#             p = [np.argmax(i.detach().numpy()) for i in p]
#             for token, label, pr in zip(w.numpy(), l.numpy(), p):
#                 if token == 0:
#                     continue
#                 news_content.append(tokenizer.index_word[token])
#                 if label == 1:
#                     true_name.append(tokenizer.index_word[token])
#                 if pr == 1:
#                     pred_name.append(tokenizer.index_word[token])

#             print('新聞內容：{}'.format(''.join(news_content)))
#             print('偵測之名字 ： {}'.format(''.join(pred_name)))
#             print('實際之名字 ： {}'.format(''.join(true_name)))
#             print('---'*30)
            
#         pred = net(inputs)
        
#         loss_fn = nn.CrossEntropyLoss()(pred, labels.view(-1)) + FocalLoss(gamma=2)(pred, labels.view(-1))
#         loss_fn = FocalLoss(gamma=5)(pred, labels.view(-1))
#         avg_val_loss += loss_fn.item()
    print('train epoch {}/{}, train loss : {:.4f} '.format(epoch+1, EPOCH, avg_loss/len(train_loader)))    
#     print('train epoch {}/{}, train loss : {:.4f}, validation loss : {:.4f}'.format(epoch+1, EPOCH, avg_loss/len(train_loader), avg_val_loss/len(test_loader)))

epoch 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


新聞內容：日前,汤唯遭遇[电信诈骗,被骗[21万余元一事,登上了微博[热门话题榜,昨日,有媒体爆出其上当细节:“汤唯当天接到‘公安’电话,
偵測之名字 ： 前汤唯被骗博汤唯
實際之名字 ： 汤唯汤唯
------------------------------------------------------------------------------------------
新聞內容：1月12日,在[加拿大温哥华,人们参加不穿裤子搭地铁活动新华社发(谢尔盖摄)全球大约[60个城市的地铁12日出现形形色色的光腿,
偵測之名字 ： 
實際之名字 ： 
------------------------------------------------------------------------------------------
新聞內容：杨文萍慰问交通局[困难职工朱桂兰1月14日,市委常委、宣传部长、[常务副市长杨文萍走访慰问[市交通局[困难职工朱桂兰,为她送去慰
偵測之名字 ： 
實際之名字 ： 杨文萍朱桂兰杨文萍朱桂兰
------------------------------------------------------------------------------------------
新聞內容：到银行存钱,还能倒贴银行钱?银行称,除扣掉每年[10元的年费,还要扣除[12元年的[小额账户管理费“12年前在银行存了[200元
偵測之名字 ： 
實際之名字 ： 
------------------------------------------------------------------------------------------
新聞內容：手枪侧卧姿射击近日,[南京军区某旅训练场上特战分队的女特种兵们正在进行自动步枪、手枪特种[射击训练。该旅的女特战队员们开训即开考
偵測之名字 ： 侧练
實際之名字 ： 
------------------------------------------------------------------------------------------
新聞內容：“工作组全跑了,我们村里的人都回去了。”武学里今天晚上在给[中国青年报记者的电话中说。离村外逃[9天之后,他终于结束了自己的流浪
偵測之名字 ： “

In [13]:
w.cpu().numpy()

array([   2,  162,   72,   40,  170,   12,   10,    3,   23,   11,    3,
         17,    3,   22,    5,    1,    2,  827,   14, 1001,  984,    1,
        827,   14,  479,   75,  300,  190,  833,   39,   49,  448,  206,
        338,  116,  417,   13,  999, 1283, 1001,  984,   20,   53,  119,
          6,   97,   42,  479,   75,  300,  190,  833,   39,  999, 1283,
       1001,  984,   15,  166,    4,    8,   70,   95,  106])